In [ ]:
import pandas as pd
import datetime
import requests
from string import Template
import pytz
import math
import logging
import time
pd.options.display.max_rows = 50
pd.options.display.max_columns = 100

# API  STUFF
API_KEY = 'qf_ZzJ56DKvoDrfdw3OVFlzGW_UfZ9el'
OPTIONS_URL = Template('https://api.polygon.io/v2/aggs/ticker/$ticker/range/$multiplier/minute/$start_date/$end_date?adjusted=true&sort=asc&limit=50000')
EQUITY_URL =  Template('https://api.polygon.io/v2/aggs/ticker/$ticker/range/$multiplier/minute/$start_date/$end_date?adjusted=true&sort=asc&limit=50000')

# logger stuff
console_logger = logging.getLogger('console_logger')
file_logger = logging.getLogger('file_logger')
file_logger.setLevel(logging.DEBUG)
console_logger.setLevel(logging.DEBUG)
file_handler = logging.FileHandler('research_options_05-11-24.log')
console_handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)
console_handler.setFormatter(formatter)
file_logger.addHandler(file_handler)
console_logger.addHandler(console_handler)
# logger stuff end

In [50]:
def getEquityQuotes(ticker: str,startDate: datetime.date,endDate: datetime.date, minutes_freq:int):
    '''
    Loop function to retrieve quotes in more than 1 request
    '''
    def _generateEquityAPIRequestQuotes(ticker: str,startDate: datetime.date,endDate: datetime.date, minutes_freq:int, nextPageUrl = False):
        headers = {
            'Authorization': f"Bearer {API_KEY}"
        }
        CSV = False
        if CSV: headers['Accept'] = 'text/csv'
        if not nextPageUrl:
            url = EQUITY_URL.substitute(
                        ticker = ticker,
                        multiplier = minutes_freq,
                        start_date = startDate,
                        end_date = endDate
                )
        else: url = nextPageUrl
        response = requests.get(url, headers=headers)
        return response
    df_responses = []
    equity_quotes = _generateEquityAPIRequestQuotes(ticker,startDate, endDate, minutes_freq)
    while True:
        if equity_quotes.status_code == 200:
                equity_df = pd.DataFrame(equity_quotes.json()['results'])
                equity_df = equity_df.rename(
                    columns={"v": "volume", "vw": "volume_weighted", "o": "open", "c": "close",
                                "h": "high", "l": "low", "t": "timestamp", "n": "number of trades",
                                "a": "trade conditions"})
                equity_df['time_converted'] = pd.to_datetime(
                    equity_df["timestamp"], unit="ms").dt.tz_localize(
                        pytz.UTC).dt.tz_convert("US/Eastern").dt.tz_localize(None)
                df_responses.append(equity_df)
                if 'next_url' in equity_quotes.json().keys():
                    # give half second break for API. Otherwise triggers error
                    time.sleep(10)
                    equity_quotes =_generateEquityAPIRequestQuotes(ticker, startDate, endDate, minutes_freq, equity_quotes.json()['next_url'])
                else: break
        else:
            print (equity_quotes.text)
            print ('Check call result, returned data that was able to retrive')
            return df_responses
            #raise RuntimeError("Error when processing API call to equity prices")
    return df_responses
equity_df_list = getEquityQuotes('QQQ',datetime.date(2023,1,1),datetime.date(2024,12,1),15)
equity_df = pd.concat(equity_df_list, ignore_index=True)

{"status":"ERROR","request_id":"f69ea10a95f1eb84d92b9393ba6e0b67","error":"You've exceeded the maximum requests per minute, please wait or upgrade your subscription to continue. https://polygon.io/pricing"}
Check call result, returned data that was able to retrive


,volume,volume_weighted,open,close,high,low,timestamp,number of trades,time_converted
0,5009787.0,475.1617,474.31,476.3900,476.90,474.120,1725047100000,37841,2024-08-30 15:45:00
1,844614.0,476.3221,476.39,475.9658,476.71,475.900,1725048000000,3104,2024-08-30 16:00:00
2,70233.0,476.1592,476.03,476.2700,476.27,475.990,1725048900000,765,2024-08-30 16:15:00
3,55583.0,476.1818,476.27,476.0900,476.27,476.040,1725049800000,285,2024-08-30 16:30:00
4,62600.0,476.1407,476.09,476.0000,476.10,475.920,1725050700000,211,2024-08-30 16:45:00
...,...,...,...,...,...,...,...,...,...
3805,31327.0,508.5127,508.40,508.5500,508.73,508.250,1732537800000,391,2024-11-25 07:30:00
3806,28344.0,508.5477,508.51,508.4400,508.66,508.440,1732538700000,312,2024-11-25 07:45:00
3807,128340.0,508.4058,508.65,508.2300,509.11,505.411,1732539600000,1388,2024-11-25 08:00:00
3808,44057.0,508.3632,508.27,508.3700,508.54,508.190,1732540500000,555,2024-11-25 08:15:00


In [57]:
equity_df = pd.concat(equity_df_list, ignore_index=True)
equity_df.to_csv('qqq_2023_2024_15_min.csv')

In [ ]:
equity_df.ta

,volume,volume_weighted,open,close,high,low,timestamp,number of trades,time_converted
30442,70764.0,25.7977,25.80,25.8899,25.89,25.72,1732581000000,397,2024-11-25 19:30:00
30443,119997.0,25.8045,25.88,25.6700,25.92,25.63,1732581900000,524,2024-11-25 19:45:00
